<a href="https://colab.research.google.com/github/mabittar/Portfolio/blob/master/MetroSP_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Metro Sao Paulo Crawler

A proposta desse notebook é criar um web crawler de maneira a registrar o status de serviço do Metrô São Paulo e armazenar os dados obtidos numa planilha do Google Drive.

<center>
<img src='https://abifer.org.br/wp-content/uploads/2017/12/metrodesaopaulo.png'>
</center>

Para atingir o objetivo iremos escrever um web scrapper capaz de receber as atualizações diretamente do site do [Metro Linha 4](http://www.viaquatro.com.br/) e extrair o status de operação.

A ideia desse post surgiu após a leitura do tópico neste [link](https://towardsdatascience.com/building-a-dataset-for-the-s%C3%A3o-paulo-subway-operation-2d8c5a430688).

Esse é apenas um exemplo de aplicação de um Crawler, que pode ser utilizado para registrar preços de um produto, oportunidades de imóveis para venda ou locação, disponibilidade de servidores e assim por diante.



## Sobre o Metrô de São Paulo

Diretamente do [site oficial](http://www.metro.sp.gov.br/) conseguimos entender um pouco mais sobre a história dessa importante companhia de transportes urbanos.

A Companhia do Metropolitano de São Paulo – Metrô foi constituída no dia 24 de abril de 1968. É controlada pelo Governo do Estado de São Paulo sob gestão da Secretaria de Estado dos Transportes Metropolitanos (STM). É responsável pela operação e expansão de rede metroviária e pelo planejamento de transporte metropolitano de passageiros da Região Metropolitana de São Paulo.

A rede metroviária da cidade de São Paulo é composta por 6 linhas, totalizando 101,1 km de extensão e 89 estações, por onde passam mais de 5 milhões de passageiros diariamente. Está integrada à CPTM nas estações Luz, Tamanduateí, Brás, Palmeiras-Barra Funda, Tatuapé, Corinthians-Itaquera, Pinheiros e Santo Amaro e aos outros modais de transporte na cidade de São Paulo.

O Metrô de São Paulo é responsável pela operação das Linhas 1-Azul (Jabaquara - Tucuruvi), 2-Verde (Vila Prudente – Vila Madalena), 3-Vermelha (Corinthians-Itaquera – Palmeiras-Barra Funda) e o Monotrilho da Linha 15-Prata (Vila Prudente – Jardim Planalto), somando 69,7 km de extensão e 62 estações. Pela rede administrada pelo Metrô, passam 4 milhões de passageiros diariamente.

## O Crawler

Web Crawler, bot ou web spider é um algoritmo utilizado para encontrar, ler e gravar informações de páginas da internet. É como um robô  que varre o caminho indicado e captura as informações que encontra pela frente.

Um dos maiores exemplos de um web crawler é o próprio google. Antes do site estar disponível para pesquisa, um robô lê o web site e cataloga as informações de forma a serem recuperadas numa busca futura.


Uma das ferramentas mais utilizadas em Python para criação de algoritmos dessa modalidade é o *BetifulSoup*. Outras informações sobre os recursos podem ser consultadas diretamente na documentação original, pelo [link](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Atenção**

Antes de iniciar o processo consulte sobre as normas do site sobre a utilização de robôs indexadores, que devem estar configuradas em www.seusite/robot.txt

O robots.txt é um arquivo deve ser salvo na pasta raiz do seu site, e indica para os robôs de busca quais as páginas de seu site você não deseja que sejam acessadas por estes mecanismos de pesquisa.

Isso ajuda a controlar o acesso de algumas informações importantes, como infográficos e detalhes técnicos de produtos.

Entretanto no site da ViaQuatro esse arquivo não foi configurado, pode ser um indicativo que o responsável pelo site não limitou o acesso a essas ferramentas.

In [46]:
# importando as bibliotecas necessárias
from bs4 import BeautifulSoup
import requests

### O alvo

No próximo passo indicamos o caminho que o robô irá buscar as informações que desejamos.

Iremos utilizar a biblioteca nativa do Python `Requests` que permite o envio de um pacote de requisição HTTP de forma extremamente fácil, mantendo a conexão ativa durante o tempo necessário de forma automática.



In [47]:
# executando o request 
home_request = requests.get('http://www.viaquatro.com.br')

# recuperando todas informações em forma de texto
home_content = home_request.text

### BeautifulSoup

Agora com a biblioteca BeatifulSoup iremos executar o `parser` da página e extraírmos as informações que desejamos.

In [48]:
soup = BeautifulSoup(home_content, 'html.parser')
#verificando o título da página
soup.title

<title>ViaQuatro | Seja Bem-vindo</title>

### Status de Operação

No passo anterior confirmarmos que a variável `soup` contém toda a página alvo do crawler. O próximo passo é executar a inspeção do site e focarmos exclusivamente na informação que desejamos obter *Status de Operação* .

Para inspecionar o site no Chrome utilize a tecla `F11` e clique no quadro que deseja, podemos ver que as informações estão dentro da classe html: class_=operacao.

Iremos direcionar nosso robô para extrair os dados dessa classe html.

In [49]:
operation_column = soup.find(class_= "operacao")
operation_column

<section class="operacao">
<!-- Current Status -->
<div class="section-title">
<h3>Operação</h3>
<p>Atualizado em <time datetime="04/12/2020 17:41">04/12/2020 17:41</time></p>
</div>
<a class="open-modal" href="#">
<div class="main-line">
<span class="circle">4</span>
<h2>Amarela</h2>
<span class="status">Operação Normal</span>
</div>
</a>
<div class="modal-situacao main">
<div class="container-modal">
<div class="box-modal station-modal">
<div class="info">
<h3>Linha Amarela</h3>
<p class="status-reduzida">Operação Normal</p>
<p></p>
<ul>
<li>
<img alt="Telefone" src="/Content/img/ico/ico-tel-black.jpg"/>
</li>
<li>
								0800-770-7100
							</li>
</ul>
</div>
</div>
</div>
</div>
<div class="title-operacao">
<h3>ViaMobilidade</h3>
</div>
<div class="linhas">
<ul>
<li>
<div class="open-modal">
<span class="circle linha-5">5</span>
<div class="info">
<a href="http://www.viamobilidade.com.br/" target="_blank" title="Acesse o site da ViaMobilidade">
<span class="title">Lilás</span>
</

Entretanto dentro dessa classe há uma série de informações que não necessitamos. Iremos criar um filtro de forma a alinhar numa lista que iremos criar o nome da linha e do filtro obter o status de operação.

In [50]:
#criando lista das linhas do Metro
linhas_metro = ['azul', 'verde', 'vermelha', 'amarela', 'lilás', 'prata']
# criando lista das linhas CPTM
linhas_cptm  = ['rubi', 'diamante', 'esmeralda', 'turquesa', 'coral', 'safira', 'jade']
# criando uma lista única
linhas = linhas_metro + linhas_cptm

#preparando a lista para obter as informações do crawler
extracted_status = {linha:'' for linha in linhas}
extracted_status

{'amarela': '',
 'azul': '',
 'coral': '',
 'diamante': '',
 'esmeralda': '',
 'jade': '',
 'lilás': '',
 'prata': '',
 'rubi': '',
 'safira': '',
 'turquesa': '',
 'verde': '',
 'vermelha': ''}

### O Filtro

Agora que temos uma lista com o nome das linhas já criado é possível filtrar das informações obtidas pelo robô o status de operação correspondente a linha definida na lista anterior.

In [51]:
lines_containers = operation_column.find_all(class_ = "linhas")

for container in lines_containers:
       line_info_divs = container.find_all(class_ = "info")
       for div in line_info_divs:
           line_title  = ''
           line_status = ''
           spans = div.find_all("span")
           line_title = spans[0].text.lower()
           line_status = spans[1].text.lower()
           extracted_status[line_title] = line_status

extracted_status

{'amarela': '',
 'azul': 'normal',
 'coral': 'normal',
 'diamante': 'normal',
 'esmeralda': 'normal',
 'jade': 'normal',
 'lilás': 'normal',
 'prata': 'normal',
 'rubi': 'normal',
 'safira': 'normal',
 'turquesa': 'normal',
 'verde': 'normal',
 'vermelha': 'normal'}

In [52]:
# Extraindo a data e horário da última atualização do site.
time_data = soup.find('time').text
time_data


'04/12/2020 17:41'

## A comemoração

Pronto! Se você seguiu os passos até aqui já está de posse de todas as informações que você necessitava.

<center><img src="https://www.juicysantos.com.br/wp-content/uploads/2017/10/cheers.jpg" width=35%></center>

[fonte](https://www.juicysantos.com.br/diversao/agenda-de-santos-e-regiao/festival-cheers-o-seu-role-nesse-feriadao/)

## A Cereja do Bolo

Até aqui tudo fácil e rápido, mas podemos ainda utilizando uma planilha do google drive armazenar as informações coletadas para estudos futuros.

Então vamos quais seriam os próximos passos?

1. Como obter as informações de acesso e autorizar o acesso a planilha?
2. O notebook irá solicitar as informações de acesso do Google Drive 
3. Escrever as informações obtidas numa planilha que estará compartilhada publicamente.


 Para a primeira etapa siga o procedimento da [documentação oficial](https://gspread.readthedocs.io/en/latest/oauth2.html):

Acesse a Página de Desenvolvedor do Google diretamente do link:
[https://console.developers.google.com/](https://console.developers.google.com/)e depois:

 - Enable API Access for a Project if you haven’t done it yet.
 - Go to “APIs & Services > Credentials” and choose “Create credentials > Service account key”.
 - Fill out the form
 - Click “Create key”
 - Select “JSON” and click “Create”

No último passo será gerado um arquivo .json para ser salvo em seu computador. Guarde esse arquivo.
Lembre-se de salvá-lo em uma pasta de fácil acesso, pois ele será utilizado para permitir o notebook acessar a planilha.  Depois criei uma nova planilha diretamente no Google Drive e compartilhe com o e-mail castrado na sua API, se você não se recorda, abra o arquivo json que você acabou de salva e busque a informação no campo client_email.

No meu caso abri uma subpasta na minha pasta de Projetos com o nome de MetroSP-Crawler, assim fica fácil de lembrar onde o arquivo está salvo.


In [ ]:
from google.colab import files
import json 
# fazendo o upload da credencial gerada anteriormente.
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# lendo as informações do arquivo
data = next(iter(uploaded.values()))
key = json.loads(data.decode())

Saving metrosp-crawler-a0056d486cda.json to metrosp-crawler-a0056d486cda.json
User uploaded file "metrosp-crawler-a0056d486cda.json" with length 2330 bytes


In [ ]:
# para facilitar iremos renomear o arquivo json para auth.json
import glob
import os

for source_name in glob.glob("/content/*.json"):
    path, fullname = os.path.split(source_name)
    basename, ext = os.path.splitext(fullname)
    target_name = os.path.join(path, '{}{}'.format('auth', ext))
    os.rename(source_name, target_name)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('auth.json', scope)

Agora vamos autorizar o acesso utilizando nossas credenciais e inicializar  a planilha.

Nesse ponto é necessário observar o id da planilha que pode ser obtido abrindo a planilha em uma outra janela e copiando do endereço do navegador o trecho equivalente destacado em negrito:
'docs.google.com/spreadsheets/d/**10MQxF_-WHXFpYaM_FJWwtPGtF3zjp2_WVnQ_OtC3pdA**/edit#gid=0

In [ ]:
#autorizando o acesso
client = gspread.authorize(creds)

#identificando a planilha
SPREADSHEET_ID = "10MQxF_-WHXFpYaM_FJWwtPGtF3zjp2_WVnQ_OtC3pdA"

# abrindo a planilha - atenção ao nome da aba que você vai usar
data_sheet = client.open_by_key(SPREADSHEET_ID).worksheet("data")


Agora que estamos com o arquivo aberto iremos escrever as informações obtidas do crawler

In [53]:
for linha in linhas:
    data_sheet.append_row([time_data, linha, extracted_status[linha]])

<img src="https://raw.githubusercontent.com/mabittar/Portfolio/master/img/Crawler-spreedsheet.jpg">